In [ ]:
# The program manager for Channel 10 would like to determine the best way to allocate the time
# for the 11:00–11:30am news program. Ten minutes is already blocked for advertising.For the
# remaining 20 minutes of broadcast time, she would like to determine the optimal amount of
# time to devote to local news, national news, weather, and sports.
# The station's broadcast policy states that at least 15% of the time available should be devoted
# to local news coverage; the time devoted to local and national news must be at least 50% of the
# available broadcast time; the time devoted to the weather segment must not exceed the time
# devoted to the sports segment; the time devoted to the sports segment should be no longer
# than the total time spent on the local and national news; and at least 20% of the available time
# should be devoted to the weather segment. The production costs per minute are $300 for local
# news, $200 for national news, $100 for weather, and $100 for sports.
# Formulate a linear program to determine the least costly program that meets all policy
# requirements.

In [1]:
%pip install -q amplpy
from amplpy import AMPL, ampl_notebook
ampl = ampl_notebook(
    modules=["highs", "cbc", "gurobi", "cplex"], # pick from over 20 modules including most commercial and open-source solvers
    license_uuid="d28bae8e-77ef-4166-9e42-b0ab0d296546") # your license UUID (e.g., free ampl.com/ce or ampl.com/courses licenses)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 34.8 MB/s eta 0:00:00
Licensed to AMPL Community Edition License for <omkarnc@umich.edu>.


In [2]:
from google.colab import files
files.upload()

Saving ampl.lic to ampl.lic


{'ampl.lic': b'# Temporary CPLEX, GUROBI, XPRESS, COPT, MOSEK, XPRESSGLOBAL, AMPL,\n# BARON, CONOPT, KNITRO, LGO, LINDOGLOBAL, LOQO, MINOS, SNOPT & OCTERACT\n# single-machine server license expiring 20251213 (CPLEX, GUROBI, XPRESS,\n# COPT, MOSEK, XPRESSGLOBAL), 20250113 (AMPL, BARON, CONOPT, KNITRO, LGO,\n# LINDOGLOBAL, LOQO, MINOS, SNOPT, OCTERACT)\n# licensed to AMPL Community Edition License for <omkarnc@umich.edu>:\n675be140 57f315c0 1b9773d8 21a2a1cf b1762fb7 3113a21b 55f3d78c 821fd316\nb7625ae5 15b0c811 79317d68 a212a6d4 b7130f6f b6e02b97 dcd3c2a2 62a48382\n2453873d 8554e06e 2f287ab5 d37c4d98 32616c19 146794e4 d93a19ad 543e2522\n4297a990 a017c2c8 13437c3f 56a90f81 b21b62d7 943568e6 ac720e41 4180eab2\n834943e6 d5312d76 fcacc4e7 f383d0e0 8692ad71 a787a64d d289ed8b f0bd0e2f\n40c5238a 778f56b1 8d46cf44 550b0a41 7316b0d0 4e5b1ba4 1d1c8c85 fcf06181\n9153cac3 6a1f9bc0 345b9522 9674449d cd56fc36 7c629fc3 966ac246 2e0326e2\nf1f34f44 aead9fb5 29c75fd4 d493ff1f b4be9066 3d088f62 43a65125 0

In [3]:
# Optionally, choose a solver (you can use 'cplex', 'gurobi', 'highs', etc.)
ampl.setOption('solver', 'highs')

In [32]:
# Clear all previous declarations
ampl.reset()

In [33]:
# Define the corrected AMPL model
ampl.eval("""
# Declare parameters
param spend_LocalNews := 300;
param spend_NationalNews := 200;
param spend_WeatherNews := 100;
param spend_SportsNews := 100;

# Declare decision variables
var l;                     # Atleast 1 Toy 1 should be produced
var n;                     # Atleast 1 Toy 2 should be produced
var w;                     # Atleast 1 Toy 2 should be produced
var s;                     # Atleast 1 Toy 2 should be produced

# Objective function: Maximize spend
minimize Total_spend: spend_LocalNews * l + spend_NationalNews * n + spend_WeatherNews * w + spend_SportsNews * s;

# Constraints
subject to Total_Time_Constraint:
    l + n + w + s == 20;

subject to LocalNews_Constraint:
    l >= 20*0.15;

subject to Local_National_News_Constraint:
    l + n >= 20*0.5;

subject to Weather_Sports_Relational_Constraint:
    w <= s;

subject to Sports_Local_National_News_Relational_Constraint:
    s <= l + n;

subject to WeatherNews_Constraint:
    w >= 20*0.20;

""")

# Solve the model
ampl.solve()

# Display results
print("Optimal Production Time of LocalNews:", ampl.getVariable('l').value())
print("Optimal Production Time of NationalNews:", ampl.getVariable('n').value())
print("Optimal Production Time of WeatherNews:", ampl.getVariable('w').value())
print("Optimal Production Time of SportsNews:", ampl.getVariable('s').value())
print("Total spend:", ampl.getObjective('Total_spend').value())

HiGHS 1.8.1: HiGHS 1.8.1: optimal solution; objective 3300
4 simplex iterations
0 barrier iterations
Optimal Production Time of LocalNews: 3.0
Optimal Production Time of NationalNews: 7.0
Optimal Production Time of WeatherNews: 4.0
Optimal Production Time of SportsNews: 6.0
Total spend: 3300.0
